# Iterative Quantum Phase Estimation (IQPE) module

The present notebook reviews the **Iterative Quantum Phase Estimation** (**IQPE**) algorithm which allows the estimation of the phase of a unitary operator without the use of the **Quantum Fourier Transform**. This **IQPE** was developed into module *iterative_quantum_pe* of the package *PE*  of the present library *QQuantLib* (**QQuantLib/PE/iterative_quantum_pe.py**). 

This **IQPE** algorithm was developed as a Python class called: *IQPE* inside the **QQuantLib/PE/iterative_quantum_pe.py**.

The present notebook and module are based on the following references:

* *Dobšíček, Miroslav and Johansson, Göran and Shumeiko, Vitaly and Wendin, Göran*. Arbitrary accuracy iterative quantum phase estimation algorithm using a single ancillary qubit: A two-qubit benchmark. Physical Review A 3(76), 2007. https://arxiv.org/abs/quant-ph/0610214

* *Griffiths, Robert B. and Niu, Chi-Sheng*. Semiclassical Fourier Transform for Quantum Computation. Physical Review Letters, 17 (76), 1996. https://arxiv.org/abs/quant-ph/9511007

* *A. Y. Kitaev*. Quantum measurements and the Abelian stabilizer problem. Electronic Colloquium on Computational Complexity, 3(3):1–22, 1996. https://arxiv.org/abs/quant-ph/9511026

* *Monz, Thomas and Nigg, Daniel and Martinez, Esteban A. and Brandl, Matthias F. and Schindler, Philipp and Rines, Richard and Wang, Shannon X. and Chuang, Isaac L. and Blatt, Rainer*. Realization of a scalable Shor algorithm. Science 6277 (351). 2016. https://arxiv.org/abs/1507.08852

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qat.lang.AQASM as qlm

In [ ]:
%matplotlib inline

### BE AWARE myqlm 1.7.1 have problems with CLinalg and Intermediate Measurements. Please DO NOT USE CLinalg (c QPU) in this notebook.

In [ ]:
#This cell loads the QLM solver. QPU = [qlmass, python, c]
from QQuantLib.utils.qlm_solver import get_qpu
QPU = ["qlmass", "python", "c"]
linalg_qpu = get_qpu(QPU[2])

## 1. Initial Inputs

For using the *IQPE* python class inside the **QQuantLib/PE/iterative_quantum_pe** module 2 mandatory QLM objects should be provided:

* 1. Initial State: this will be the initial quantum state needed for applying the Unitary Operator.
* 2. Unitary operator: the operator whose phase we want to estimate.

To explain how the *IterativeQuantumPE* class works we are going to use the **IQPE** example from the Qiskit textbook:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb

We are going to reproduce the section **IPE example with a 1-qubit gate for U** from the Qiskit example

In [ ]:
#Number Of Qbits
n_qbits = 1

### 1. Initial State

Initial State can be:
1. QLM QRoutine
2. QLM gate (or abstract gate)

In the Qiskit example, the initial state will be $|1\rangle$. 

The following cell creates this initial state:

In [ ]:
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

### 2. Unitary operator.

The unitary operator can be:

1. QLM QRoutine
2. QLM gate (or abstract gate)

In the Qiskit example, the unitary operator is the $\mathcal{S}$ gate. So the application of the unitary operator over the initial state will be:

$$\mathcal{S}|1\rangle = e^{i\frac{\pi}{2}}|1\rangle$$

In [ ]:
unitary_operator = qlm.PH(np.pi/2.0)  

In [ ]:
%qatdisplay unitary_operator --svg

## 2. Class IQPE: algorithm step by step 

The problem of phase estimation can be stated as follows. Given an initial state $\left|\Psi \right\rangle$ and a phase operator $\mathcal{P}$ such that:

$$\mathcal{P}\left|\Psi \right\rangle = e^{2\pi i\lambda}\left|\Psi \right\rangle,$$

our goal is estimating $\lambda$.

So far we have the initial state $\left|\Psi \right\rangle = |1\rangle$ and the unitary operator whose phase we want to estimate $\mathcal{P} = \mathcal{S}$. In this section, we are going to describe the class step by step and explain the basics of the **IPE** algorithm

### 2.1 Calling the **IQPE** class

The *IterativeQuantumPE* is inside **QQuantLib/PE/iterative_quantum_pe** module. 

To instantiate the class we need to provide a Python dictionary. Mandatory keys are:

* initial_state : QLM routine or gate with an initial state $|\Psi\rangle$ was loaded. 
* unitary_operator :  QLM gate or routine with a Unitary operator ready to be applied to initial state $|\Psi\rangle$.

Other important keys are:

* cbits_number : int with the number of classical bits needed for phase estimation
* qpu : QLM solver. If not provided class try to create a PyLinalg solver. It is recommended to give this key to the class.
* shots : int number of shots for the quantum job.

In [ ]:
#Load Class
from QQuantLib.PE.iterative_quantum_pe import IQPE

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
}
iqpe_object = IQPE(**iqpe_dict)

When the class is instantiated the properties *initial_state* and *q_gate* are overwritten with the given keys **initial_state** and **unitary_operator** respectively

In [ ]:
c = iqpe_object.initial_state
%qatdisplay c --svg

In [ ]:
a = iqpe_object.q_gate 
%qatdisplay a --depth 2 --svg

### 2.2 IPE Algorithm step by step

Now we are going to review step by step the **IPE** algorithm using different programmed methods of the **IterativeQuantumPE** class

#### 2.2.1. Initialize the quantum program.

The first thing is calling the method **init_iqpe**. The following actions are done by this method:
1. Creation of QLM program from *initial_state* QLM routine (or AbstractGate). The QLM program is stored in the *q_prog* property.
2. Allocation of an auxiliary qubit is mandatory for the **IPE** algorithm. It is stored in the *q_aux* property.
3. Allocation of the auxiliary classical bits where the estimated phase will be stored. Property: *c_bits*.

In [ ]:
#Initialize the quantum program
iqpe_object.init_iqpe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliary qubit bits was allocated
circuit = iqpe_object.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 0 --svg

#### 2.2.2. IQPE Algorithm

We are going to decompose the **IQPE** algorithm in 2 parts. A first part where the main variable $l$ will be 0 ($l=0$) and a second recursive part where the variable $l$ will be greater than 0 ($l\gt 0$).

#### First Part ($l=0$)

The first step of the IPE algorithm ($l=0$) has the following parts:

1. Reset the auxiliary qubit
2. Apply a Hadamard gate to the auxiliary qubit
3. Apply the phase operator $\mathcal{P}$ controlled by auxiliary the qubit $2^{m-1}$ ($\mathcal{P} ^{2^{m-1}}$) times. Here $m$ is the number of classical qubits allocated for estimating $\theta$ 
4. Apply a Hadamard gate to the auxiliary qubit
5. Measuring the auxiliary qubit and storing the result into a classical bit array in position $c_l$ ($l=0$)

This can be done by calling the *step_iqpe* method with the following arguments:

* Quantum Program with initial_state
* Quantum Routine or AbstractGate with phase operator $\mathcal{P}$
* Auxiliary Qubit
* Auxiliary classical bits
* l=0

This *step_iqpe* method returns the quantum program with the operations explained in this part.

In [ ]:
q_program = iqpe_object.q_prog
q_program = iqpe_object.step_iqpe(
    q_program, 
    iqpe_object.q_gate,
    iqpe_object.q_aux,
    iqpe_object.c_bits, 
    l=0
)

Following cell show the first  part of the circuit

In [ ]:
c = q_program.to_circ()
%qatdisplay c --svg

#### Second or iterative Part ($l \gt 0$)

This part of the algorithm is recursive. The following steps will be repeated for each value of $l=1,2,..m-1$ (m is the number of classical bits for codifying the phase)

1. Reset the auxiliary qubit
2. Apply a Hadamard gate to the auxiliary qubit
3. Apply the phase operator $\mathcal{P}$ controlled by the auxiliary qubit $2^{m-1-l}$ ($\mathcal{Q}^{2^{m-1-l}}$) times, being $m$ is the number of classical bits allocated for estimating $\theta$.
4. Apply on the auxiliary qubit a set of controlled rotations by $c_j$ classical bit of angle: $\frac{\pi}{2}\frac{1}{2^{l-j-1}}$ with $j=0,1,..l-1$. 
4. Apply a Hadamard gate to the auxiliary qubit
5. Measuring the auxiliary qubit and storing the result into a classical bit array in position $c_l$

So for a $l$ step de controlled by classical bits rotation will depend on the measurements done on the before $l$ steps.

In the following cell we explain how to create the algorithm for the $l=1$ part:

In [ ]:
#here we do the step l=1
l=1
q_program = iqpe_object.step_iqpe(
    q_program, 
    iqpe_object.q_gate, 
    iqpe_object.q_aux,
    iqpe_object.c_bits, 
    l=l)

Now we can plot the circuit we have until the moment

In [ ]:
#Circuit for l=0 and l=1
c = q_program.to_circ()
%qatdisplay c --svg

####  Complete algorithm

For a complete **IPE** algorithm following steps should be done:

1. Create the First part of the algorithm $l=0$.
2. Iterate the second part of the algorithm from  $l=1$ to $l=m-1$ where m is the number of classical bits for estimating the phase

The measured classical bits are used for estimating the phase of the unitary operator

The following cell creates the complete program for the **IQPE** algorithm

In [ ]:
#Initialize the quantum program
iqpe_object.init_iqpe()
q_program = iqpe_object.q_prog
for l in range(len(iqpe_object.c_bits)):
    q_program = iqpe_object.step_iqpe(
        q_program, 
        iqpe_object.q_gate,
        iqpe_object.q_aux,
        iqpe_object.c_bits,
        l=l)

So for the desired number of classical bits $m$ the complete circuit for *IPE* algorithm will be:

In [ ]:
c = q_program.to_circ()
%qatdisplay c --svg

#### 2.2.3. IQPE Algorithm execution

Once the QLM program is constructed the algorithm should be executed. For this the *run* method from the class allow to execute it. The following arguments should  be provided:

* q_prog : with the complete *IPE* algorithm
* q_aux : the auxiliary qubit 
* shots 
* linalg_qpu: QLM solver

This method creates the circuit, and the associated job and executes it. The raw results of the simulation and the circuit are returned


In [ ]:
raw_results, circuit = iqpe_object.run_qprogram(
    q_program,
    iqpe_object.q_aux, 100, linalg_qpu=linalg_qpu)

In [ ]:
%qatdisplay circuit --svg

#### 2.2.4. IPE: getting classical bits measurements

As explained the phase will be estimated by getting the measurements of the classical bits. In the class, this is done by the **measure_classical_bits** method. The input of this method is the *raw_results* from the *run_qprogram* method. The output will be a pandas DataFrame with the aggregated measurement of the classical bits with the following columns:

* **BitString**: is the result of the classical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as $\frac{BitInt}{2^{m}}$ where $m$ is the number of classical bits used for phase estimation
* **Frequency**: frequency of the obtained **BitString** (number of times the **BitString** divide by the number of shots)

In [ ]:
classical_bits = iqpe_object.measure_classical_bits(raw_results)

In [ ]:
classical_bits

From *classical_bits* pdf the important column is **Phi**. Following the Qiskit example, this column is $\varphi$ and the searched phase is: $\phi=2\pi\varphi$. In the Qiskit example $\varphi=0.25$. 

The *classical_bits* DataFrame gives the aggregated results for all executions of the circuit (variable *shots*). 

#### 2.2.5. IPE: post-processing

Typically the phase in radians is provided in this kind of phase estimation problem. To get this result we use the *post_proccess*. The *classical_bits* should be provided and the output will be another DataFrame with the following columns:

* **BitString**: is the result of the classical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as $\frac{BitInt}{2^{c_b}}$ where $c_b$ is the number of classical bits 
* **2*theta**: this is the phase of the unitary operator in radians: $2\theta = 2\pi\varphi$. In this case, we calculate the phase of the unitary operator as the double of an angle $\theta$.
* **theta**: this is the half of the phase of the unitary operator: $\theta = \pi\varphi$
* **theta_90**: is the $\theta$ between $(0, \frac{\pi}{2}$)
* **Frequency**: Frequency of each **BitString**

The returned DataFrame from *post_proccess* was ordered by decreasing values of the frequency (so the first row is for the most frequent **BitString**, the second row for the second most frequent value and so on)

In this module our convention when $|\Psi\rangle $ is an eigenvalue of an unitary operator $\mathcal{Q}$ is:

$$\mathcal{Q}|\Psi\rangle = e^{2i\theta}|\Psi\rangle$$

In [ ]:
final_results = iqpe_object.post_proccess(classical_bits)

In [ ]:
final_results

### 2.3. Class IQPE: complete execution

In section 2.2 the complete algorithm step by step was explained. The methods shown in this sub-section were shown only for pedagogical purposes but users **SHOULD NOT** use these methods. Instead of these methods user **SHOULD USE** the **iqpe** where all the steps explained in section 2.2 are executed. When using this *method* the following properties are populated:


* *classical_bits*: the DataFrame with the result of the *measure_classical_bits* method.
* *final_results*: the DataFrame with the result of the *post_proccess* method.

Additionally, the **iqpe** method overwrites the *pdf_time* property of the class where several elapsed times are stored

In [ ]:
n_cbits = 2
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_object = IQPE(**iqpe_dict)
iqpe_object.iqpe()

In [ ]:
#classical_bits attribute
iqpe_object.classical_bits

In [ ]:
#final_results attribute
iqpe_object.final_results

The attribute circuit of the class stores the qlm circuit created

In [ ]:
c = iqpe_object.circuit
%qatdisplay c --svg

## Another Example

We are going to reproduce now the Qiskit example under the section *IPE example with a 2-qubit gate* (in https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb). 

In this case, they use 2 qubits and want to estimate the phase for a unitary operator $\mathcal{cT}$ operator. this operator adds a $\frac{\pi}{4}$ phase to state $|11\rangle$ and leave unchanged other states. 


In [ ]:
#create initial state
n_qbits = 2
initial_state = qlm.QRoutine()
q_bits = initial_state.new_wires(n_qbits)
for i in range(n_qbits):
    initial_state.apply(qlm.X, q_bits[i])

In [ ]:
%qatdisplay initial_state --svg

In [ ]:
#Create cT operator
unitary_operator = qlm.QRoutine()
uq_qbits = unitary_operator.new_wires(n_qbits)
unitary_operator.apply(qlm.PH(np.pi/4.0).ctrl(), 0, 1)

In [ ]:
%qatdisplay unitary_operator --svg

In [ ]:
#now IPE!!
n_cbits = 3
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': initial_state,
    'unitary_operator': unitary_operator,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_object = IQPE(**iqpe_dict)

In [ ]:
iqpe_object.iqpe()

In [ ]:
iqpe_object.classical_bits

The result for $\varphi=0.125$ that is the value obtained in the Qiskit example.

In [ ]:
c = iqpe_object.circuit
%qatdisplay c --svg

In [ ]:
iqpe_object.final_results

## 3. Application to Amplitude Estimation

The problem of amplitude estimation is the following. Given an oracle:

$$\mathcal{0}|0\rangle = |\Psi\rangle = \sqrt{a}|\Psi_0\rangle +\sqrt{1-a}|\Psi_1\rangle,$$

where $|\Psi_0\rangle$ and $|\Psi_1\rangle$ are orthogonal states, we want to estimate $\sqrt{a}$.


The *Iterative Quantum Phase Estimation* can be applied to *Amplitude Estimation* problem. The present section explains how we can take advantage of our **IQPE** class for this.

First, we will define the following amplitude estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{1}|1\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    & \sqrt{a}|\Psi_0\rangle \longrightarrow \dfrac{\sqrt{1}}{\sqrt{0+1+2+3+4+5+6+7+8}}|1\rangle.\\
    & \sqrt{1-a}|\Psi_1\rangle \longrightarrow \scriptstyle \dfrac{1}{\sqrt{0+1+2+3+4+5+6+7+8}}\left[\sqrt{0}|0\rangle+\sqrt{2}|2\rangle+\sqrt{3}|3\rangle+\sqrt{4}|4\rangle+\sqrt{5}|5\rangle+\sqrt{6}|6\rangle+\sqrt{7}|7\rangle\right].\\
    \end{array}
$$

The following cells create the initial state $|\Psi\rangle$

In [ ]:
from QQuantLib.DL.data_loading import load_probability
from QQuantLib.AA.amplitude_amplification import grover
from QQuantLib.PE.iterative_quantum_pe import IQPE

In [ ]:
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

Next, we will show how the Phase Estimation problem relates to the Amplitude Estimation problem:
$$
    \begin{array}{l}
    & |\Psi\rangle \longrightarrow |\Psi\rangle\\
    & \mathcal{P} \longrightarrow \mathcal{G}
    \end{array}
$$
The first equation means that, in the phase estimation context, the initial state is $|\Psi\rangle$ and the phase operator is $\mathcal{G}$, the Grover operator corresponding to our amplitude estimation problem. 

In the next cell, we define the Grover operator for our problem.  

In [ ]:
target = [0, 0, 1]
index = range(oracle.arity)
grover_gate = grover(oracle, target, index)
#Comment before line and uncomment following for For multiplexor implementation of multi-controlled Z gate
#grover_gate = grover(oracle, target, index, mcz_qlm=False)
%qatdisplay grover_gate --depth  --svg

Here we have used that our target state $|1\rangle$ in binary representation is $001$. See notebook *02_AmplitudeAmplification_Operators* for more information about building Grover operators.

Now that we have translated our amplitude amplification problem to a phase estimation problem we proceed to use our class normally. We provide the *oracle* as the **initial_state** and the correspondent Grover-like operator as the **unitary_operator**. Additionally, the number of classical bits (**cbits_number**) for estimating the phase should be provided.

In [ ]:
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict = {
    'initial_state': oracle,
    'unitary_operator': grover_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,  
    'shots': 100
}
iqpe_object = IQPE(**iqpe_dict)
iqpe_object.iqpe()

In [ ]:
iqpe_object.final_results

In [ ]:
#We can plot the obtained angles vs the its frequency
plt.plot(
    iqpe_object.final_results['theta_90'], 
    iqpe_object.final_results['Frequency'], 'o')
plt.xlabel(r'$\theta$')
plt.xlim(0, 0.5*np.pi)
plt.ylabel('Frecuency')
plt.title(r'$\theta \in [0, \frac{\pi}{2}]$')

Last, we use the mapping $a = \cos(\theta)^2$ to obtain the result of our amplitude estimation problem.

In [ ]:
theta_estimated = iqpe_object.final_results['theta_90'].iloc[0]
print('Quantum result, theta_estimated: ', theta_estimated)
a_estimated = np.cos(theta_estimated)**2
print('Quantum result, a_estimated: ', a_estimated)
print("Classical result: ",probability[1])

In [ ]:
print('Test OK: ', abs(probability[1]- a_estimated)< 0.005)

In [ ]:
c = iqpe_object.circuit
%qatdisplay c --svg --depth 0

In order to increase the precision of the angle estimation more classical bits can be used

## 4. Amplitude Estimation with IQPE

In order to simplify **Amplitude Estimation** calculations done with **IQPE** (see Section 3) the **IQPEAE** (stands for **I**terative **Q**uantum **P**hase **E**stimation **A**mplitude **E**stimation) class was created. The idea behind this class is dealing with the initial overheating shown in Section 3  for using **IQPE** in a straightforward way for *Amplitude Estimation* calculations. The class is under the module **ae_iterative_quantum_pe.py** of the *Amplitude Estimation* package of the *QQuantLib* library  (**QQuantLib/AE/ae_iterative_quantum_pe**)

This new class try to reproduce the scheme of the **MLAE class** from **QQuantLib/AE/maximum_likelihood_ae.py** module

To create **IQPEAE** object following inputs are mandatory:

1. Oracle: QLM AbstractGate or QRoutine with the implementation of the Oracle for creating the Grover operator.
2. target: this is the marked state in binary representation as a Python list
3. index: list of the qubits affected by the Grover operator.

In [ ]:
from QQuantLib.DL.data_loading import load_probability

In [ ]:
#Here we created the mandatory oracle
n = 3
N = 2**n
x = np.arange(N)
probability = x/np.sum(x)
oracle = load_probability(probability)

%qatdisplay oracle --depth 0 --svg

#This will be the target state for grover and the list of qubits affected by Grover operator
target = [0, 0, 1]
index = range(oracle.arity)

Now we have all mandatory inputs so we created the *IQPEAE* object. Following **MLAE class** convention other parameters can be provided with a python dictionary. In this case, the following keys can be provided:

* *auxiliar_qbits_number*: number of qubits for doing phase estimation (default: 8)
* *shots*: number of shots (default: 100)
* *qpu*: qpu solver
* mcz_qlm: for using QLM multi-controlled Z gate (True, default) or using multiplexor implementation (False)

In [ ]:
from QQuantLib.AE.ae_iterative_quantum_pe import IQPEAE

In [ ]:
ae_iqpe_dict = {
    'qpu': linalg_qpu,
    'cbits_number': 4,
    'shots': 10,
    'mcz_qlm': True      
}

ae_iqpe = IQPEAE(
    oracle=oracle,
    target=target,
    index=index, 
    **ae_iqpe_dict
)

When instantiated the **IQPEAE** the *Grover* operator associated to the oracle operator is created. It can be access using attribute *_grover_oracle*.

In [ ]:
grover_circ = ae_iqpe._grover_oracle
%qatdisplay grover_circ --depth --svg

Now the *ae_iqpe* has all mandatory inputs for calling the **IQPE** class and executing it. The **run** method from **AE_PhaseEstimationwQFT** class does this step. Additionally when the *run* method is executed following properties from **PE_QFTAE** class are populated:

* *iqpe_object*: this property is an object from the **IQPE** class that was initialized with corresponding arguments created by the **IQPEAE** class.
* *final_results*: this is the final_results obtained from the *iqpe* method of the property object *iqpe_object* (see section 2.3).
* *theta*: estimated $\theta$ obtained from **IQPE** algorithm.
* *ae*: estimated solution of the *Amplitude Amplification* problem using **IQPE** algorithm $a=cos^2 \theta$
* *run_time*: elapsed time of a complete **run** method

In [ ]:
a_estimated = ae_iqpe.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_iqpe.final_results= ', ae_iqpe.final_results)
print('ae_iqpe.theta= ', ae_iqpe.theta)
print('ae_iqpe.a= ', ae_iqpe.ae)
print("Classical result: ",probability[1])

In [ ]:
print('Test OK: ',np.abs(ae_iqpe.ae-probability[1])< 0.005)

In [ ]:
print("Elapsed time of a run method: ", ae_iqpe.run_time)

Of course the configuration of the algorithm can be changed by setting the attribute of the class an calling the *run* method again!!

In [ ]:
ae_iqpe.cbits_number = 10
print(ae_iqpe.cbits_number)

In [ ]:
a_estimated = ae_iqpe.run()

In [ ]:
print('a_estimated = ',a_estimated) 
print('ae_pe_withQFT.theta= ', ae_iqpe.theta)
print('ae_pe_withQFT.a= ', ae_iqpe.ae)
print("Classical result: ",probability[1])

In [ ]:
print("Elapsed time of a run method: ", ae_iqpe.run_time)

The **run** method of the *IQPEAE* creates and configures an object from *IQPE* class. This object is stored on the **pe_iqpe** attribute of the *IQPEAE*. All the attributes and methods from the *IQPE* class can be accessed from this **pe_iqpe** attribute. For example, we can get the complete quantum circuit executed!!!

As explained before, the **iqpe_object** attribute from the *IQPEAE* class is an object created from class **IQPE** so all the methods and attributes of this class can be accessed using the **iqpe_object** attribute (**NOT RECOMMENDED**).

We can access the quantum circuit used by the **IQPE**

In [ ]:
#First we access to the atribute tha stores the IterativeQuantumPE object
attribute_ae_iqpe = ae_iqpe.iqpe_object

#Then we take the circuit attribute from the PhaseEstimationwQFT object (see above in the notebook)
c=attribute_ae_iqpe.circuit
%qatdisplay c --svg --depth 0

When the *run* method is executed following class attributes are populated:

* *circuit_statistics*: python dictionary with the complete statistical information of the circuit.
* *oracle_calls*: number of total oracle calls for a complete execution of the algorithm
* *max_oracle_depth*: maximum number of applications of the oracle for the complete execution of the algorithm.

In [ ]:
ae_iqpe.circuit_statistics

In [ ]:
#Total number of oracle calls
print("The total number of the oracle calls for the IQPEAE was: {}".format(ae_iqpe.oracle_calls))

In [ ]:
#Number of maximum oracle applications
ae_iqpe.max_oracle_depth

In [ ]:
ae_iqpe.quantum_times

In [ ]:
ae_iqpe.quantum_time